In [1]:
# Step 1: Import Necessary Libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import time
from sklearn.utils.class_weight import compute_class_weight

# --- Constants and Configuration ---
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 20
DATA_DIR = '/Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/'
CLASS_NAMES = ['cataract', 'conjunctivitis', 'eyelid', 'normal', 'uveitis']

In [ ]:

# OLD Step 2: Load and Preprocess the Data
def load_data(data_dir, class_names, img_width, img_height):
    images = []
    labels = []
    print("Loading image data...")
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, (img_width, img_height))
                images.append(img)
                labels.append(class_name)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    print("Image data loaded successfully.")
    return np.array(images), np.array(labels)


In [4]:
# NEW Step 2: Load and Preprocess the Data
def load_data(data_dir, class_names, img_width, img_height):
    images = []
    labels = []
    print("Loading image data...")
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            try:
                img = cv2.imread(img_path)

                # --- THIS IS THE FIX ---
                # Convert from BGR (OpenCV) to RGB (Matplotlib)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                # --- END OF FIX ---

                img = cv2.resize(img, (img_width, img_height))
                images.append(img)
                labels.append(class_name)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    print("Image data loaded successfully.")
    return np.array(images), np.array(labels)

# Load the initial data
images, labels = load_data(DATA_DIR, CLASS_NAMES, IMG_WIDTH, IMG_HEIGHT)

# Normalize pixel values
images = images.astype('float32') / 255.0

# --- Model Definition ---
def create_ovr_model(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)):
    """Creates a binary classifier based on EfficientNetB0 for OvR."""
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(name="avg_pool"),
        layers.BatchNormalization(),
        layers.Dropout(0.3, name="top_dropout"),
        # Final layer for binary classification (One-vs-Rest)
        layers.Dense(1, activation='sigmoid', name="pred")
    ])
    return model

Loading image data...
Image data loaded successfully.


In [ ]:
# OLD Step 4: Display sample images
def show_sample_images(sample_images, sample_labels, class_names_map):
    plt.figure(figsize=(12, 120))
    for i in range(min(30, len(sample_images))): # Show up to 240 images
        plt.subplot(60, 4, i + 1)
        plt.imshow(sample_images[i])
        # Convert one-hot encoded label back to integer, then to class name
        label_index = sample_labels[i]
        plt.title(class_names_map[label_index])
        plt.axis("off")
    plt.suptitle("Sample Images from Training Set", fontsize=16, y=0.9)
    plt.show()

# Display some images from the training set to verify


In [5]:
# Step 4: Display sample images
def show_sample_images(sample_images, sample_labels, class_names_map):
    plt.figure(figsize=(12, 120))
    for i in range(min(30, len(sample_images))): # Show up to 240 images
        plt.subplot(60, 4, i + 1)
        plt.imshow(sample_images[i])

        # --- START OF FIX ---
        label_value = sample_labels[i]

        # Check if the label is an integer (like 0, 1) or a string (like 'cataract')
        if isinstance(label_value, (int, np.integer)):
            # If it's an integer, use it as an index to look up the name
            plt.title(class_names_map[label_value])
        else:
            # If it's already a string, just use it directly
            plt.title(label_value)
        # --- END OF FIX ---

        plt.axis("off")
    plt.suptitle("Sample Images from Training Set", fontsize=16, y=0.9)
    plt.show()

In [ ]:

# OLD--- One-vs-Rest (OvR) Training Loop ---
# This loop will train a separate binary classifier for each class.

for class_name in CLASS_NAMES[0:1]:
    print("\n" + "="*50)
    print(f"    Training Model for: {class_name} vs. Rest")
    print("="*50 + "\n")

    # 1. Create Binary Labels for the current class
    # The "positive" class is labeled 1, all others are 0.
    print(f"Preparing binary labels for '{class_name}'...")
    binary_labels = np.array([1 if label == class_name else 0 for label in labels])

    # 2. Split the Data for the current binary classification task
    # We stratify to ensure the train/val/test sets have a similar proportion of positive/negative samples.
    (X_train_temp, X_test, y_train_temp, y_test) = train_test_split(
        images, binary_labels, test_size=0.20, stratify=binary_labels, random_state=42
    )
    (X_train, X_val, y_train, y_val) = train_test_split(
        X_train_temp, y_train_temp, test_size=0.25, stratify=y_train_temp, random_state=42 # 0.25 * 0.8 = 0.2
    )
    print(f"Data split complete for '{class_name}'.")
    print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}, Testing samples: {len(X_test)}")

show_sample_images(images, labels, CLASS_NAMES)
    # 3. Create and Compile the Model
    # We use 'binary_crossentropy' for the loss function.
    ovr_model = create_ovr_model()
    ovr_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    print(f"Model for '{class_name}' compiled successfully.")

    # 4. Train the Model
    print(f"Starting training for '{class_name}' for {EPOCHS} epochs...")
    start_time = time.time()
    history = ovr_model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        verbose=2 # Set to 2 for less output per epoch
    )
    end_time = time.time()
    print(f"Training for '{class_name}' finished in {end_time - start_time:.2f} seconds.")

    # 5. Save the Trained Model
    # Each model is saved with a unique name.
    model_filename = f"efficientnet_ovr_{class_name}.h5"
    ovr_model.save(model_filename)
    print(f"Model saved to {model_filename}")

    # 6. Evaluate the Model on the Test Set
    test_loss, test_acc = ovr_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {class_name} vs. Rest: {test_acc:.2%}")
    print(f"Test Loss for {class_name} vs. Rest: {test_loss:.4f}")
    print("\nDisplaying a sample of training images...")

print("\n" + "="*50)
print("All One-vs-Rest models have been trained successfully!")
print("="*50 + "\n")

In [ ]:
print(X_train, y_train, X_val, y_val, X_test, y_test)
print(CLASS_NAMES)
show_sample_images(X_train, y_train, ["target", "other"])


In [ ]:
# NEW--- One-vs-Rest (OvR) Training Loop ---
# This version includes class weights to handle data imbalance and a two-stage fine-tuning process.

INITIAL_EPOCHS = 10
FINE_TUNE_EPOCHS = 10
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS

for class_name in CLASS_NAMES:
    print("\n" + "="*50)
    print(f"    Training Model for: {class_name} vs. Rest")
    print("="*50 + "\n")

    # 1. Create Binary Labels
    binary_labels = np.array([1 if label == class_name else 0 for label in labels])

    # 2. Split the Data
    (X_train_temp, X_test, y_train_temp, y_test) = train_test_split(
        images, binary_labels, test_size=0.20, stratify=binary_labels, random_state=42
    )
    (X_train, X_val, y_train, y_val) = train_test_split(
        X_train_temp, y_train_temp, test_size=0.25, stratify=y_train_temp, random_state=42
    )
    print(f"Data split complete for '{class_name}'.")

    # --- THE FIX: CALCULATE CLASS WEIGHTS ---
    # This balances the loss function, forcing the model to pay attention to the minority class.
    weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight = {0: weights[0], 1: weights[1]}
    print(f"Class weights for '{class_name}': {class_weight}")

    # 3. Create and Compile the Model for Stage 1
    ovr_model = create_ovr_model()
    ovr_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # --- Stage 1: Initial Training (with Class Weights) ---
    print("\n--- Starting Stage 1: Initial Training ---")
    history = ovr_model.fit(
        X_train, y_train,
        epochs=INITIAL_EPOCHS,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        class_weight=class_weight,  # <--- Apply the class weights here
        verbose=2
    )

    # --- Stage 2: Fine-Tuning ---
    print("\n--- Starting Stage 2: Fine-Tuning ---")
    ovr_model.layers[0].trainable = True # Unfreeze the base model

    # Unfreeze the top 30 layers for fine-tuning
    for layer in ovr_model.layers[0].layers[:-30]:
        layer.trainable = False

    # Re-compile with a very low learning rate for fine-tuning
    ovr_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Continue training (fine-tuning)
    history_fine = ovr_model.fit(
        X_train, y_train,
        epochs=TOTAL_EPOCHS,
        initial_epoch=history.epoch[-1],
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        class_weight=class_weight, # <--- Also apply weights during fine-tuning
        verbose=2
    )

    # 4. Save and Evaluate the Final Model
    model_filename = f"efficientnet_ovr_{class_name}_finetuned.h5"
    ovr_model.save(model_filename)
    print(f"Model saved to {model_filename}")

    test_loss, test_acc = ovr_model.evaluate(X_test, y_test, verbose=0)
    print(f"\nFinal Test Accuracy for {class_name} vs. Rest: {test_acc:.2%}")

print("\nAll models have been trained and fine-tuned successfully!")

In [ ]:

# --- Inference Section: Making a Prediction on a New Image ---

def predict_image_ovr(image_path, model_dir='.'):
    """
    Loads a new image, preprocesses it, and predicts its class
    using the saved One-vs-Rest models.
    """
    # 1. Load the trained OvR models
    ovr_models = {}
    for class_name in CLASS_NAMES:
        model_path = os.path.join(model_dir, f"efficientnet_ovr_{class_name}.h5")
        if os.path.exists(model_path):
            ovr_models[class_name] = tf.keras.models.load_model(model_path)
        else:
            print(f"Warning: Model not found for class '{class_name}' at {model_path}")
            return

    # 2. Load and preprocess the new image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img_array = np.expand_dims(img, axis=0)  # Add batch dimension
    img_array = img_array.astype('float32') / 255.0

    # 3. Get a prediction score from each binary classifier
    scores = {}
    for class_name, model in ovr_models.items():
        score = model.predict(img_array)[0][0]
        scores[class_name] = score

    # 4. Determine the final prediction
    # The class with the highest probability score is the winner.
    predicted_class = max(scores, key=scores.get)
    highest_score = scores[predicted_class]

    print("\n--- Prediction Results ---")
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Predicted Condition: {predicted_class.capitalize()}")
    print(f"Confidence Score: {highest_score:.2%}")
    print("\n--- Individual Model Scores ---")
    for class_name, score in sorted(scores.items(), key=lambda item: item[1], reverse=True):
        print(f"- {class_name.capitalize()}: {score:.2%}")

# Example of how to use the prediction function
# Make sure you have an image to test with, for example 'test_image.jpg'
predict_image_ovr('/Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/img_1.png')